In [9]:
import numpy as np
import pandas as pd
import os, glob

In [13]:
# Importing relevant files:
projDir = os.environ['HOME'] + '/Dropbox/Projects/mc/'
dataDir = projDir + 'data/'
subjDirs = glob.glob(os.path.join(dataDir, 'mcvct*'))